# Data Mining Project - ABCDEatsInc.

**Group 16 members:** <br>
- Ana Margarida Valente, 20240936 <br>
- Catarina Carneiro, 20240690 <br>
- Rui Reis, 20240854 <br>
- Mara Mesquita, 20241039 <br>

**MSc:** Data Science and Advanced Analytics - Nova IMS <br>
**Course:** Data Mining <br>
2024/2025

## Introduction
The client, "ABCDEatsInc.", a fictional food delivery service partnering with a range of restaurants to offer diverse meal options, aims to gain a deeper understanding of its customers' behaviors by identifying distinct segments within its database through customer segmentation. A dataset containing data collected over three months from three cities was provided to define, describe, and analyze the resulting clusters. The goal is to uncover actionable insights and briefly recommend marketing strategies tailored to each segment. 


## Variables:
- customer_id: Unique identifier for each customer.
- customer_region: Geographic region where the customer is located.
- customer_age: Age of the customer.
- vendor_count: Number of unique vendors the customer has ordered from.
- product_count: Total number of products the customer has ordered.
- is_chain: Indicates whether the customer’s order was from a chain restaurant.
- first_order: Number of days from the start of the dataset when the customer first placed an order.
- last_order: Number of days from the start of the dataset when the customer most recently placed an order.
- last_promo: The category of the promotion or discount most recently used by the customer.
- payment_method: Method most recently used by the customer to pay for their orders.
- CUI_American,CUI_Asian,CUI_Chinese,CUI_Italian, etc.: The amount in monetary units spent by the customer from the indicated type of cuisine.
- DOW_0 to DOW_6: Number of orders placed on each day of the week (0 = Sunday, 6 = Saturday).
- HR_0 to HR_23: Number of orders placed during each hour of the day (0 = midnight, 23 = 11 PM).

# Table of Contents !!!

<a class="anchor" id="import">

## 1. Import 
    
</a>


<a class="anchor" id="libraries">

## 1.1 Import libraries
    
</a>

In [2]:
import sqlite3
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil

from itertools import product
from scipy.stats import skewnorm

from datetime import datetime
from sklearn.impute import KNNImputer

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

from sklearn.preprocessing import MinMaxScaler, StandardScaler

# for better resolution plots
%config InlineBackend.figure_format = 'retina' # optionally, you can change 'svg' to 'retina'

# Setting seaborn style
sns.set()

# Display all the df and results
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None)  
pd.set_option('display.width', 1000)  
pd.set_option('display.colheader_justify', 'center')  

<a class="anchor" id="dataset">

## 1.2 Import the dataset
    
</a>

In [3]:
# Read the CSV file
df = pd.read_csv("DM2425_ABCDEats_DATASET.csv")

In [4]:
#Create a backup of the original df
df_backup=df

<a class="anchor" id="exploration">

# 2. Data Exploration
    
</a>

<a class="anchor" id="general">

## 2.1 General Data Analysis
    
</a>

In [ ]:
df.shape

In [ ]:
# Display the first few rows of the dataframe
df.head()

In [ ]:
#Check columns
df.columns.values

In [ ]:
#Check Data Types
df.dtypes

In [ ]:
# Summary Statistics of the Categorical variables
df.describe(include='O').T

In [ ]:
# Summary Statistics of the Numerical variables
df.describe().T


In [ ]:
# Check duplicates
df.isna().any()

In [ ]:
df.isna().sum()

In [ ]:
#Check the % of the missing values:
missing_percentage = (df.isnull().mean() * 100).sort_values(ascending=False)

print("Percentage of Missing Values:")
print(missing_percentage)

<a class="anchor" id="treat">

## 2.2 Data Treatment
    
</a>

<a class="anchor" id="rename">

### 2.2.1 Rename Columns
    
</a>

- Change the DOW columns to the days of the week names

In [14]:
df= df.rename(columns={'DOW_0':'Sunday', 'DOW_1':'Monday', 'DOW_2':'Tuesday', 'DOW_3':'Wednesday', 'DOW_4':'Thursday', 'DOW_5':'Friday','DOW_6':'Saturday'})

<a class="anchor" id="types">

### 2.2.2 Data Types
    
</a>

- customer_age -> float? (change to int)
- first_order -> float? (change to int)
- HR_0 -> float? (change to int)

In [15]:
df['customer_age'] = df['customer_age'].astype('Int64')
df['first_order'] = df['first_order'].astype('Int64')
df['HR_0'] = df['HR_0'].astype('Int64')

<a class="anchor" id="nan">

### 2.2.3 Missing Values
    
</a>

**Missing Values : HR_0**

In [16]:
# #Check when HR_0 = NaN, which is the variable with most missing values
# nan_HR_0 = df[df['HR_0'].isna()]
# pd.set_option('display.max_columns', None)
# nan_HR_0

In [17]:
#Define the columns of the DOW and the HR columns
dow_columns = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
hr_columns = [col for col in df.columns if col.startswith('HR_')]

In [ ]:
#To replace the NaN's of HR_0, let's compare the sum of orders of the DOW with the sum of orders of the HR:
#If it's the same, then HR_0 should be 0, if not, it's the difference between the 2 values, since the sum should be equal

row_sum_dow = df[dow_columns].sum(axis=1)
row_sum_hr = df[hr_columns].sum(axis=1)

row_difference = row_sum_dow - row_sum_hr

df.loc[df['HR_0'].isna(), 'HR_0'] = row_difference

df['HR_0'].head()

In [ ]:
df['HR_0'].isna().sum()

In [ ]:
#Check if the sum of the DOW is equal to the sum of the Hours; It must be
check = (df[dow_columns].sum(axis=1) == df[hr_columns].sum(axis=1)).all()

if check:
    print("Yes")
else:
    print("No")

**Missing Values : first_order**

In [21]:
# #Check when first_order = NaN
# nan_first_order = df[df['first_order'].isna()]
# nan_first_order

It seem that when the first_order is a missing value, the last_order = 0.

In [22]:
# df[df['first_order'].isna() & (df['last_order'] == 0)]

In [ ]:
#First we are checking to see if in this situation, there was only one order placed
check = (df[df['first_order'].isna() & (df['last_order'] == 0)][dow_columns].sum(axis=1) == 1).all()

if check:
    print("All rows have row_sum_dow equal to 1 (indicating only one order).")
else:
    print("There are rows where row_sum_dow is not 1.")

In [ ]:
#Check which rows do not meet the condition
non_matching_rows = df[(df['first_order'].isna() & (df['last_order'] == 0)) & (df[dow_columns].sum(axis=1) != 1)]

non_matching_rows

There are only 2 cases that do not meet the condition. Both cases show that the 2 orders were placed on the same day (Saturday). 

Based on the previous analysis, we will assume that when first_order is missing it should be replaced with 0, ensuring that both first_order and last_order occur on the same day (the day the dataset begun).

In [25]:
#first_order is missing only when last_order = 0
#first_order cannot happen after last_order. So we will set the missing first_order values to 0
df.loc[df['first_order'].isna() & (df['last_order'] == 0), 'first_order'] = 0

**Missing Values : customer_age**

In [26]:
# # Check if there are any missing values
# if df['customer_age'].isnull().any():
    
#     missing_values_before = df.loc[df['customer_age'].isnull(), ['customer_age']].copy()


#     imputer = KNNImputer(n_neighbors=5, weights="uniform")

    
#     df['customer_age'] = imputer.fit_transform(df[['customer_age']])

#     df['customer_age'] = df['customer_age'].round().astype(int)

#     # Create a DataFrame to compare the original values (NaN) with the imputed values
#     comparison = pd.DataFrame({
#         'Original_Index': missing_values_before.index,
#         'Original_Value': missing_values_before['customer_age'].values,
#         'Imputed_Value': df['customer_age'].loc[missing_values_before.index].values
#     })

#     print(comparison)
# else:
#     print("No missing values found in 'customer_age'.")

In [27]:
# #Check when customer_age = NaN
# nan_customer_age = df[df['customer_age'].isna()]
# nan_customer_age

# #Replace the missing values with the mean or median

In [ ]:
plt.hist(df['customer_age'].dropna(), bins=30, edgecolor='black')  
plt.title('Histogram of Age')
plt.xlabel('Age') 
plt.ylabel('Frequency') 
plt.show()

In [ ]:
#Since the histogram is skewed, the median is preferred to replace the missing values
median_age = df['customer_age'].median()
df['customer_age'] = df['customer_age'].fillna(median_age)
print(f"Median = {median_age}")

<a class="anchor" id="duplicates">

### 2.2.4 Check for Duplicates
    
</a>

In [ ]:
df.duplicated().sum()

In [ ]:
df.loc[df.duplicated(keep=False)]

In [ ]:
#% of duplicates:
df.duplicated().mean()*100

In [ ]:
#Identify customer_id duplicated (it should be a unique value, since it represents 1 customer)
duplicate_values = df['customer_id'].value_counts()[df['customer_id'].value_counts() > 1]

duplicate_rows = df[df['customer_id'].isin(duplicate_values.index)]

print("\n")
print(f'Total: {len(duplicate_rows)}') 
duplicate_rows

The duplicates are only the cases where the customer_id is duplicated, meaning that are 2 entries of the same customer in the dataset

In [34]:
df.drop_duplicates(inplace=True)

In [35]:
df = df.set_index('customer_id')

In [ ]:
df.head()

<a class="anchor" id="uniqueandstrange">

### 2.2.5 Check Unique and Strange values
    
</a>

**Vendor_count**

In [ ]:
df['vendor_count'].unique()

In [ ]:
df[df['vendor_count'] == 41]

**Product_count**

In [ ]:
df['product_count'].unique()

In [ ]:
df[df['product_count'] == 269]

**Customer_region**

In [ ]:
df['customer_region'].value_counts()

In [ ]:
(len(df[df['customer_region'] == '-']) / len(df))*100

In [43]:
# Replace '-' with the mode
mode_value = df['customer_region'].mode()[0]  

df['customer_region'] = df['customer_region'].replace('-', mode_value)

- '-' -> Strange = 1,386%; Assuming it's a missing value and Use Mode
- There are 3 cities, aggregate the cities by the first number of the region (2,4,8)

**Customer_age**

In [ ]:
df['customer_age'].unique()

In [ ]:
df[(df['customer_age'] == 15) | (df['customer_age'] == 16)| (df['customer_age'] == 17)]
#It could be a problem since it's a minor

**last_promo**

In [ ]:
df['last_promo'].value_counts()

- '-' -> Changing to 'NO PROMO', to be more perceptible

In [47]:
df['last_promo'] = df['last_promo'].replace('-', 'NO PROMO')

**Payment_method**

In [ ]:
df['payment_method'].value_counts()

**First_order**

In [ ]:
df['first_order'].unique()

In [ ]:
df['first_order'].max()
#Makes sense because the dataset is from a three-month period

**Last_order**

In [ ]:
df['last_order'].unique()

In [ ]:
df['first_order'].max()

**is_chain**

This variable needs to be fixed. The metadata does not correspond to the dataset

In [ ]:
df['is_chain'].unique()

In [ ]:
# Most of the orders are on bevarages
df[df['is_chain'] == 83]

Change the is_chain variable to Binary Type

In [55]:
threshold = 0
df['is_chain'] = (df['is_chain'] > threshold).astype(int)

In [ ]:
df['is_chain']

**DOW**

In [57]:
# for column in dow_columns:
#     unique_values = df[column].unique()  # Get unique values
#     print(f"Column: {column}")
#     print(f"Unique Values: {unique_values}")
#     print("-" * 70)

**Hours**

In [58]:
# for column in hr_columns:
#     unique_values = df[column].unique()  # Get unique values
#     print(f"Column: {column}")
#     print(f"Unique Values: {unique_values}")
#     print("-" * 70)

In [ ]:
df[df['HR_8'] == 52] #Almost every purchase was on Beverages (possibly for breakfast?)

**Cuisine Types**

In [60]:
# for col in df.columns:
#     if col.startswith('CUI_'):
#         unique_values = df[col].unique()  
#         print(f"Column: {col}")
#         print(f"Unique Values: {unique_values}")
#         print("-" * 70)

<a class="anchor" id="visual">

# 3. Data Visualization
    
</a>

 Define Numerical and Categorical Features

In [61]:
numerical_features=['customer_age', 'vendor_count','product_count', 'first_order', 'last_order']
categorical_features=['customer_region','last_promo','payment_method','is_chain']

<a class="anchor" id="num">

## 3.1 Numerical Features
    
</a>

In [62]:
def analyze_outliers(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_lim = Q1 - 1.5 * IQR
    upper_lim = Q3 + 1.5 * IQR
    outliers = data[(data < lower_lim) | (data > upper_lim)]
    percentage = (len(outliers) / len(data)) * 100  
    return len(outliers), percentage, lower_lim, upper_lim

In [ ]:
for col in numerical_features:
    print(f" Statistics for column: {col}")
    
    # Calculate key statistics
    mean = df[col].mean()
    median = df[col].median()
    std_dev = df[col].std()
    min_val = df[col].min()
    max_val = df[col].max()
    skewness = df[col].skew()
    kurtosis = df[col].kurt()

    # Display the statistics
    print(f'  Mean: {mean:.2f}')
    print(f'  Median: {median:.2f}')
    print(f'  Standard Deviation: {std_dev:.2f}')
    print(f'  Min: {min_val}')
    print(f'  Max: {max_val}')
    print(f'  Skewness: {skewness:.2f}')
    print(f'  Kurtosis: {kurtosis:.2f}')
    print('-' * 50 )
    
    # Visualization of each Variable:
    plt.figure(figsize=(12, 5))

    # Histogram
    plt.subplot(1, 2, 1)
    sns.histplot(df[col], kde=True, color='steelblue')
    plt.title(f'Histogram of {col}')

    # Boxplot
    plt.subplot(1, 2, 2)
    sns.boxplot(y=df[col], color='steelblue')
    plt.title(f'Boxplot of {col}')

    #set fixed scale between -10 and 100 (to analyse better the outliers)
    plt.ylim(-10,100)

    # Analyze outliers for the numerical variables
    outlier_count, outlier_percentage, lower_lim, upper_lim = analyze_outliers(df[col])
  
    print(f'Count of outliers: {outlier_count}')
    print(f'Percentage of outliers: {outlier_percentage:.2f}%')
    print(f'Lower Lim:{lower_lim}')
    print(f'Upper Lim:{upper_lim}')
    print('-' * 40)
   
    plt.tight_layout()

In [64]:
# cui_bx=['CUI_American', 'CUI_Asian', 'CUI_Beverages','CUI_Cafe', 'CUI_Chicken Dishes', 
#         'CUI_Chinese', 'CUI_Desserts','CUI_Healthy', 'CUI_Indian', 'CUI_Italian', 
#         'CUI_Japanese','CUI_Noodle Dishes', 'CUI_OTHER', 'CUI_Street Food / Snacks','CUI_Thai']

# for col in cui_bx:
#     print(f" Statistics for column: {col}")
    
#     # Calculate key statistics
#     mean = df[col].mean()
#     median = df[col].median()
#     std_dev = df[col].std()
#     min_val = df[col].min()
#     max_val = df[col].max()
#     skewness = df[col].skew()
#     kurtosis = df[col].kurt()

#     # Display the statistics
#     print(f'  Mean: {mean:.2f}')
#     print(f'  Median: {median:.2f}')
#     print(f'  Standard Deviation: {std_dev:.2f}')
#     print(f'  Min: {min_val}')
#     print(f'  Max: {max_val}')
#     print(f'  Skewness: {skewness:.2f}')
#     print(f'  Kurtosis: {kurtosis:.2f}')
#     print('-' * 50 )
    
#     # Visualization of each Variable:
#     plt.figure(figsize=(12, 5))

#     # Histogram
#     plt.subplot(1, 2, 1)
#     sns.histplot(df[col], kde=True, color='steelblue')
#     plt.title(f'Histogram of {col}')

#     # Boxplot
#     plt.subplot(1, 2, 2)
#     sns.boxplot(y=df[col], color='steelblue')
#     plt.title(f'Boxplot of {col}')

#     #set fixed scale between -10 and 250 (to analyse better the outliers)
#     plt.ylim(-10, 250)

#     # Analyze outliers for the numerical variables
#     outlier_count, outlier_percentage, lower_lim, upper_lim = analyze_outliers(df[col])
  
#     print(f'Count of outliers: {outlier_count}')
#     print(f'Percentage of outliers: {outlier_percentage:.2f}%')
#     print(f'Lower Lim:{lower_lim}')
#     print(f'Upper Lim:{upper_lim}')
#     print('-' * 40)
   
#     plt.tight_layout()
#     plt.show()

In [65]:
# hr_bx= ['HR_0', 'HR_1', 'HR_2', 'HR_3', 'HR_4', 'HR_5','HR_6', 'HR_7', 'HR_8', 'HR_9', 'HR_10', 
#         'HR_11', 'HR_12', 'HR_13','HR_14', 'HR_15', 'HR_16', 'HR_17', 'HR_18', 'HR_19', 'HR_20', 'HR_21','HR_22', 'HR_23']

# for col in hr_bx:
#     print(f" Statistics for column: {col}")
    
#     # Calculate key statistics
#     mean = df[col].mean()
#     median = df[col].median()
#     std_dev = df[col].std()
#     min_val = df[col].min()
#     max_val = df[col].max()
#     skewness = df[col].skew()
#     kurtosis = df[col].kurt()

#     # Display the statistics
#     print(f'  Mean: {mean:.2f}')
#     print(f'  Median: {median:.2f}')
#     print(f'  Standard Deviation: {std_dev:.2f}')
#     print(f'  Min: {min_val}')
#     print(f'  Max: {max_val}')
#     print(f'  Skewness: {skewness:.2f}')
#     print(f'  Kurtosis: {kurtosis:.2f}')
#     print('-' * 50 )
    
#     # Visualization of each Variable:
#     plt.figure(figsize=(12, 5))

#     # Histogram
#     plt.subplot(1, 2, 1)
#     sns.histplot(df[col], kde=True, color='steelblue')
#     plt.title(f'Histogram of {col}')

#     # Boxplot
#     plt.subplot(1, 2, 2)
#     sns.boxplot(y=df[col], color='steelblue')
#     plt.title(f'Boxplot of {col}')

#     #set fixed scale between -10 and 20 (to analyse better the outliers)
#     plt.ylim(-10,20)
    
#     # Analyze outliers for the numerical variables
#     outlier_count, outlier_percentage, lower_lim, upper_lim = analyze_outliers(df[col])
  
#     print(f'Count of outliers: {outlier_count}')
#     print(f'Percentage of outliers: {outlier_percentage:.2f}%')
#     print(f'Lower Lim:{lower_lim}')
#     print(f'Upper Lim:{upper_lim}')
#     print('-' * 40)
   
#     plt.tight_layout()
#     plt.show()

In [66]:
# dow_bx=['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday','Friday', 'Saturday']

# for col in dow_bx:
#     print(f" Statistics for column: {col}")
    
#     # Calculate key statistics
#     mean = df[col].mean()
#     median = df[col].median()
#     std_dev = df[col].std()
#     min_val = df[col].min()
#     max_val = df[col].max()
#     skewness = df[col].skew()
#     kurtosis = df[col].kurt()

#     # Display the statistics
#     print(f'  Mean: {mean:.2f}')
#     print(f'  Median: {median:.2f}')
#     print(f'  Standard Deviation: {std_dev:.2f}')
#     print(f'  Min: {min_val}')
#     print(f'  Max: {max_val}')
#     print(f'  Skewness: {skewness:.2f}')
#     print(f'  Kurtosis: {kurtosis:.2f}')
#     print('-' * 50 )
    
#     # Visualization of each Variable:
#     plt.figure(figsize=(12, 5))

#     # Histogram
#     plt.subplot(1, 2, 1)
#     sns.histplot(df[col], kde=True, color='steelblue')
#     plt.title(f'Histogram of {col}')

#     # Boxplot
#     plt.subplot(1, 2, 2)
#     sns.boxplot(y=df[col], color='steelblue')
#     plt.title(f'Boxplot of {col}')

#     #set fixed scale between -10 and 20 (to analyse better the outliers)
#     plt.ylim(-10, 20)

#     # Analyze outliers for the numerical variables
#     outlier_count, outlier_percentage, lower_lim, upper_lim = analyze_outliers(df[col])
  
#     print(f'Count of outliers: {outlier_count}')
#     print(f'Percentage of outliers: {outlier_percentage:.2f}%')
#     print(f'Lower Lim:{lower_lim}')
#     print(f'Upper Lim:{upper_lim}')
#     print('-' * 40)
   
#     plt.tight_layout()
#     plt.show()

In [67]:
#Define groups of columns of the same category
dow_columns = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
hr_columns = [col for col in df.columns if col.startswith('HR_')]
CUI_columns = [col for col in df.columns if col.startswith('CUI_')]
#Define the sum of the columns
DOW_counts = df[dow_columns].sum()
HR_counts = df[hr_columns].sum()
CUI_counts = df[CUI_columns].sum()

In [ ]:
plt.figure(figsize=(12, 6))
DOW_counts.plot(kind='bar', 
                color='steelblue', 
                edgecolor='black')

plt.title('Number of Orders for Each Day of the Week')
plt.xlabel('Day of the Week')
plt.ylabel('Number of Orders')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
HR_counts.plot(kind='bar', 
               color='steelblue', 
               edgecolor='black')

plt.title('Number of Orders for Each Hour')
plt.xlabel('Hours')
plt.ylabel('Number of Orders')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
CUI_counts_sorted = CUI_counts.sort_values(ascending=False)

plt.figure(figsize=(12, 6))
CUI_counts_sorted.plot(kind='bar', 
                       color='steelblue', 
                       edgecolor='black')

plt.title('Expenses for each Type of Cuisine')
plt.xlabel('Type of Cuisine')
plt.ylabel('Expenses')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
sns.pairplot(df, vars= numerical_features)

In [ ]:
sns.regplot(x=df['vendor_count'], y=df['product_count'], color='steelblue')
plt.show()

<a class="anchor" id="cat">

## 3.2 Categorical Features
    
</a>

In [ ]:
for col in categorical_features:
    print(f"Analysis for categorical column: {col}")
    
    # Calculate frequency counts
    freq_counts = df[col].value_counts()
    
    # Display the frequency counts
    print("Frequency counts:")
    print(freq_counts)
    print("-"*50 )
    
    # Visualization for categorical variables:
    plt.figure(figsize=(8, 4))
    sns.countplot(x=df[col], palette='tab10')
    plt.title(f'Count Plot of {col}')
    plt.xlabel(col)
    plt.ylabel('Count')
    plt.show()

<a class="anchor" id="out">

# 4. Outliers
    
</a>

<a class="anchor" id="outanalysis">

## 4.1 Analysis
    
</a>


In [74]:
#Define groups of columns of the same category
dow_columns = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
hr_columns = [col for col in df.columns if col.startswith('HR_')]
CUI_columns = [col for col in df.columns if col.startswith('CUI_')]

#Define the sum of the columns
DOW_counts = df[dow_columns].sum()
HR_counts = df[hr_columns].sum()
CUI_counts = df[CUI_columns].sum()

In [75]:
combined_list_numerical = numerical_features + hr_columns + dow_columns + CUI_columns 

In [ ]:
#computing the interquartile range 
q1= df[combined_list_numerical].quantile(0.25)
q3=df[combined_list_numerical].quantile(0.75)
iqr=q3-q1

#compute the limits 
lower_lim= q1-(1.5*iqr)
upper_lim=q3+(1.5*iqr)

for feature in combined_list_numerical: 
    print(f"{feature:<25} Lower Limit:{lower_lim[feature]:>10}      Upper Limit:{upper_lim[feature]:>10}")

In [ ]:
def identify_outliers(df, list, lower_lim, upper_lim):
    outliers = {}
    obvious_outliers = []

    for metric in combined_list_numerical:
        if metric not in df.columns:
            continue
        
        if metric not in lower_lim.keys() or metric not in upper_lim.keys():
            continue
        
        outliers[metric] = []
        llim = lower_lim[metric]
        ulim = upper_lim[metric]
        
        for i, value in enumerate(df[metric]):
            if pd.isna(value):
                continue
            
            if value < llim or value > ulim:
                outliers[metric].append(value)
        
        print(f"Total outliers in {metric}: {len(outliers[metric])}")

    # Check for observations that are outliers in all features (Obvious Outliers)
    for index, row in df.iterrows():
        is_global_outlier = True
        for metric in combined_list_numerical:
            if metric not in df.columns or metric not in lower_lim or metric not in upper_lim:
                is_global_outlier = False
                break
            
            value = row[metric]
            if pd.isna(value):
                is_global_outlier = False
                break
            
            llim = lower_lim[metric]
            ulim = upper_lim[metric]
            
            if llim <= value <= ulim:
                is_global_outlier = False
                break
        
        if is_global_outlier:
            obvious_outliers.append(index)
    print("-----------------------------")
    print(f"Total global outliers: {len(obvious_outliers)}")
    return outliers, obvious_outliers
    
    
outliers, obvious_outliers = identify_outliers(df, combined_list_numerical, lower_lim, upper_lim)

Since we know that aren't outliers in the variables first_order and last_order, let's remove this features and analyse if there are any Total global outliers

In [ ]:
combined_list_numerical_ = [col for col in combined_list_numerical if col not in ['first_order', 'last_order']]

outliers, obvious_outliers = identify_outliers(df, combined_list_numerical_, lower_lim, upper_lim)

In [79]:
# filter that will verify if an observation has every characteristic in the Interquartile Range or not 
filters_iqr = []                                            
for metric in combined_list_numerical:
    llim = lower_lim[metric]
    ulim = upper_lim[metric]
    filters_iqr.append(df[metric].between(llim, ulim, inclusive='neither'))

filters_iqr_all = pd.concat(filters_iqr, axis=1).all(axis=1)

In [ ]:
filters_iqr

In [ ]:
filters_iqr_all  

In [ ]:
len(df[df[filters_iqr_all] == 'False'])      #These are the features that have at least one of its characteristics considered as an outlier (out of the IQR)

In [ ]:
df_iqr = df[filters_iqr_all]
print('Percentage of data kept after removing outliers:', 100*(np.round(df_iqr.shape[0] / df.shape[0], decimals=2)))

<a class="anchor" id="remove">

## 4.2 Manual Removal
    
</a>

In [84]:
filters_man = (
     (df['customer_age']<=80)  
    &
    (df['vendor_count']<=40)
    &
    (df['product_count']<=80)  
    &
    (df['is_chain']<=60)
    &
    (df['HR_0']<=10)
    &
    (df['HR_1']<=10)
    &
    (df['HR_2']<=10)  
    &
    (df['HR_3']<=10) 
    &
    (df['HR_4']<=10) 
    &
    (df['HR_5']<=10) 
    &
    (df['HR_6']<=10) 
    &
    (df['HR_7']<=10) 
    &
    (df['HR_8']<=15) 
    &
    (df['HR_9']<=15) 
    &
    (df['HR_10']<=15) 
    &
    (df['HR_11']<=15) 
    &
    (df['HR_12']<=15) 
    &
    (df['HR_13']<=15) 
    &
    (df['HR_14']<=10) 
    &
    (df['HR_15']<=10) 
    &
    (df['HR_16']<=15) 
    &
    (df['HR_17']<=15) 
    &
    (df['HR_18']<=15) 
    &
    (df['HR_19']<=15) 
    &
    (df['HR_20']<=10) 
    &
    (df['HR_21']<=10) 
    &
    (df['HR_22']<=10) 
    &
    (df['HR_23']<=10) 
    &
    (df['CUI_American']<=150)
    &
    (df['CUI_Asian']<=200)
    &
    (df['CUI_Beverages']<=150)  
    &
    (df['CUI_Cafe']<=150) 
    &
    (df['CUI_Chicken Dishes']<=50) 
    &
    (df['CUI_Chinese']<=100) 
    &
    (df['CUI_Desserts']<=100) 
    &
    (df['CUI_Healthy']<=150) 
    &
    (df['CUI_Indian']<=150) 
    &
    (df['CUI_Italian']<=150) 
    &
    (df['CUI_Japanese']<=150) 
    &
    (df['CUI_Noodle Dishes']<=100) 
    &
    (df['CUI_OTHER']<=200) 
    &
    (df['CUI_Street Food / Snacks']<=200) 
    &
    (df['CUI_Thai']<=150)
    & 
    (df['Sunday']<=15)
    &
    (df['Monday']<=15)
    &
    (df['Tuesday']<=15)  
    &
    (df['Wednesday']<=15) 
    &
    (df['Thursday']<=15) 
    &
    (df['Friday']<=15) 
    &
    (df['Saturday']<=15)  
)

df_outliers = df[filters_man]


In [ ]:
print('Percentage of data kept after removing outliers:', 100*(np.round(df_outliers.shape[0] / df.shape[0], decimals=5)))

<a class="anchor" id="combo">

## 4.3 Combining different outlier methods
    
</a>

In [ ]:
df_out = df[(filters_iqr_all | filters_man)] 


print('Percentage of data kept after removing outliers:', np.round(df_out.shape[0] / df.shape[0], 4))

In [87]:
# Get the manual filtering version

df = df_outliers.copy()

<a class="anchor" id="featengi">

# 5. Feature Engineering
    
</a>

  - Time Periods

In [88]:
#Time Periods
df['early_morning(0h-5h)'] = df.filter(regex=r'^HR_[0-5]$').sum(axis=1).astype(int)

df['morning(6h-11h)'] = df.filter(regex=r'^HR_([6-9]|1[0-1])$').sum(axis=1).astype(int)

df['afternoon(12h-17h)']= df.filter(regex=r'^HR_1[2-7]$').sum(axis=1).astype(int)

df['night(18h-23h)'] = df.filter(regex=r'^HR_(1[8-9]|2[0-3])$').sum(axis=1).astype(int)

time_columns = ['early_morning(0h-5h)','morning(6h-11h)','afternoon(12h-17h)', 'night(18h-23h)']
time_counts = df[time_columns].sum()

In [ ]:
plt.figure(figsize=(12, 6))
time_counts.plot(kind='bar', 
                 color='steelblue', 
                 edgecolor='black')

plt.title('Number of Orders for Time Period')
plt.xlabel('Time Period')
plt.ylabel('Number of Ordes')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

  - Age Group

In [90]:
#Age Group
age_labels = ['Teenagers (15-19)', 'Young Adults (20-29)', 'Adults (30-49)', 'Middle-aged (50-64)', 'Seniors (65+)']
df['age_group'] = pd.cut(df['customer_age'], 
                         bins=[15, 20, 30, 50, 65, np.inf], 
                         labels=age_labels, 
                         right=False)

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(x='age_group', data=df, color='steelblue',edgecolor='black')

plt.title('Count of Age Groups', fontsize=16)
plt.xlabel('Age Group', fontsize=12)
plt.ylabel('Count', fontsize=12)

plt.tight_layout()
plt.show()

- Sum of Orders

In [92]:
#Define a new Feature that contains the sum of orders by customer (it has the same info as DOW_counts and HR_counts)
df['Sum_of_Orders'] = df[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']].sum(axis=1)

In [ ]:
col='Sum_of_Orders'
# Calculate key statistics
mean = df[col].mean()
median = df[col].median()
std_dev = df[col].std()
min_val = df[col].min()
max_val = df[col].max()
skewness = df[col].skew()
kurtosis = df[col].kurt()

# Display the statistics
print(f'  Mean: {mean:.2f}')
print(f'  Median: {median:.2f}')
print(f'  Standard Deviation: {std_dev:.2f}')
print(f'  Min: {min_val}')
print(f'  Max: {max_val}')
print(f'  Skewness: {skewness:.2f}')
print(f'  Kurtosis: {kurtosis:.2f}')
print('-' * 50 )
    
# Visualization of each Variable
plt.figure(figsize=(12, 5))

# Histogram
plt.subplot(1, 2, 1)
sns.histplot(df[col], bins=10,color='steelblue' )
plt.title(f'Histogram of {col}')

# Boxplot
plt.subplot(1, 2, 2)
sns.boxplot(y=df[col], color='steelblue')
plt.title(f'Boxplot of {col}')

# Analyze outliers for the specified numerical variables
outlier_count, outlier_percentage, lower_lim, upper_lim = analyze_outliers(df[col])
  
print(f'Count of outliers: {outlier_count}')
print(f'Percentage of outliers: {outlier_percentage:.2f}%')
print(f'Lower Lim:{lower_lim}')
print(f'Upper Lim:{upper_lim}')
print('-' * 40)
   
plt.tight_layout()
plt.show()

- Recency

In [94]:
# Determine the maximum number of days (most recent day in dataset)
max_days = df['last_order'].max()

# Calculate recency
df['recency'] = max_days - df['last_order']

In [ ]:
col='recency'
# Calculate key statistics
mean = df[col].mean()
median = df[col].median()
std_dev = df[col].std()
min_val = df[col].min()
max_val = df[col].max()
skewness = df[col].skew()
kurtosis = df[col].kurt()

# Display the statistics
print(f'  Mean: {mean:.2f}')
print(f'  Median: {median:.2f}')
print(f'  Standard Deviation: {std_dev:.2f}')
print(f'  Min: {min_val}')
print(f'  Max: {max_val}')
print(f'  Skewness: {skewness:.2f}')
print(f'  Kurtosis: {kurtosis:.2f}')
print('-' * 50 )
    
# Visualization of each Variable
plt.figure(figsize=(12, 5))

# Histogram
plt.subplot(1, 2, 1)
sns.histplot(df[col], bins=10, color='steelblue')
plt.title(f'Histogram of {col}')

# Boxplot
plt.subplot(1, 2, 2)
sns.boxplot(y=df[col], color='steelblue')
plt.title(f'Boxplot of {col}')

# Analyze outliers for the specified numerical variables
outlier_count, outlier_percentage, lower_lim, upper_lim = analyze_outliers(df[col])
  
print(f'Count of outliers: {outlier_count}')
print(f'Percentage of outliers: {outlier_percentage:.2f}%')
print(f'Lower Lim:{lower_lim}')
print(f'Upper Lim:{upper_lim}')
print('-' * 40)
   
plt.tight_layout()
plt.show()

- Frequency

In [96]:
# Calculate active period
df['active_period'] = df['last_order'] - df['first_order'] + 1

# Calculate frequency
df['frequency'] = df['Sum_of_Orders'] / df['active_period']

In [ ]:
col='frequency'
# Calculate key statistics
mean = df[col].mean()
median = df[col].median()
std_dev = df[col].std()
min_val = df[col].min()
max_val = df[col].max()
skewness = df[col].skew()
kurtosis = df[col].kurt()

# Display the statistics
print(f'  Mean: {mean:.2f}')
print(f'  Median: {median:.2f}')
print(f'  Standard Deviation: {std_dev:.2f}')
print(f'  Min: {min_val}')
print(f'  Max: {max_val}')
print(f'  Skewness: {skewness:.2f}')
print(f'  Kurtosis: {kurtosis:.2f}')
print('-' * 50 )
    
# Visualization of each Variable
plt.figure(figsize=(12, 5))

# Histogram
plt.subplot(1, 2, 1)
sns.histplot(df[col], bins=10, color='steelblue')
plt.title(f'Histogram of {col}')

# Boxplot
plt.subplot(1, 2, 2)
sns.boxplot(y=df[col], color='steelblue')
plt.title(f'Boxplot of {col}')

# Analyze outliers for the specified numerical variables
outlier_count, outlier_percentage, lower_lim, upper_lim = analyze_outliers(df[col])
  
print(f'Count of outliers: {outlier_count}')
print(f'Percentage of outliers: {outlier_percentage:.2f}%')
print(f'Lower Lim:{lower_lim}')
print(f'Upper Lim:{upper_lim}')
print('-' * 40)
   
plt.tight_layout()
plt.show()

- Monetary 

In [98]:
# Total spend per customer
df['total_spend'] = df[CUI_columns].sum(axis=1)

In [ ]:
col='total_spend'
# Calculate key statistics
mean = df[col].mean()
median = df[col].median()
std_dev = df[col].std()
min_val = df[col].min()
max_val = df[col].max()
skewness = df[col].skew()
kurtosis = df[col].kurt()

# Display the statistics
print(f'  Mean: {mean:.2f}')
print(f'  Median: {median:.2f}')
print(f'  Standard Deviation: {std_dev:.2f}')
print(f'  Min: {min_val}')
print(f'  Max: {max_val}')
print(f'  Skewness: {skewness:.2f}')
print(f'  Kurtosis: {kurtosis:.2f}')
print('-' * 50 )
    
# Visualization of each Variable
plt.figure(figsize=(12, 5))

# Histogram
plt.subplot(1, 2, 1)
sns.histplot(df[col], bins=10, color='steelblue')
plt.title(f'Histogram of {col}')

# Boxplot
plt.subplot(1, 2, 2)
sns.boxplot(y=df[col], color='steelblue')
plt.title(f'Boxplot of {col}')

# Analyze outliers for the specified numerical variables
outlier_count, outlier_percentage, lower_lim, upper_lim = analyze_outliers(df[col])
  
print(f'Count of outliers: {outlier_count}')
print(f'Percentage of outliers: {outlier_percentage:.2f}%')
print(f'Lower Lim:{lower_lim}')
print(f'Upper Lim:{upper_lim}')
print('-' * 40)
   
plt.tight_layout()
plt.show()

RFM 
- Recency = 'recency'
- Frequency = 'frequency'
- moentary = 'total_spend'

- Cuisine Diversity 

We decided to create a new variable called cuisine_diversity to measure the variety of cuisines each customer orders from. This variable will help us analyze which age groups or regions/cities tend to explore a wider range of cuisines, indicating openness to new experiences. Conversely, it will allow us to identify customers who stick to fewer options, showing a strong preference for specific types of cuisine 

In [100]:
# Cuisine diversity (number of different cuisines ordered)
df['cuisine_diversity'] = (df[CUI_columns] > 0).sum(axis=1)

In [ ]:
col='cuisine_diversity'
# Calculate key statistics
mean = df[col].mean()
median = df[col].median()
std_dev = df[col].std()
min_val = df[col].min()
max_val = df[col].max()
skewness = df[col].skew()
kurtosis = df[col].kurt()

# Display the statistics
print(f'  Mean: {mean:.2f}')
print(f'  Median: {median:.2f}')
print(f'  Standard Deviation: {std_dev:.2f}')
print(f'  Min: {min_val}')
print(f'  Max: {max_val}')
print(f'  Skewness: {skewness:.2f}')
print(f'  Kurtosis: {kurtosis:.2f}')
print('-' * 50 )
    
# Visualization of each Variable
plt.figure(figsize=(12, 5))

# Histogram
plt.subplot(1, 2, 1)
sns.histplot(df[col], bins=10, color='steelblue')
plt.title(f'Histogram of {col}')

# Boxplot
plt.subplot(1, 2, 2)
sns.boxplot(y=df[col], color='steelblue')
plt.title(f'Boxplot of {col}')

# Analyze outliers for the specified numerical variables
outlier_count, outlier_percentage, lower_lim, upper_lim = analyze_outliers(df[col])
  
print(f'Count of outliers: {outlier_count}')
print(f'Percentage of outliers: {outlier_percentage:.2f}%')
print(f'Lower Lim:{lower_lim}')
print(f'Upper Lim:{upper_lim}')
print('-' * 40)
   
plt.tight_layout()
plt.show()

- City
  
Aggregate Regions by the First Digit, which indicates the City

In [ ]:
# Creating new Feature customer_city dividied into categories 
def categorize_city(customer_region):
    if pd.isna(customer_region):  # Check if the value is NaN
        return np.nan 
    elif customer_region== "-":
        return "Other"
    elif customer_region[0].isdigit():
        return customer_region[0]
    else:
        return "Other"


# Apply the function to create the new 'customer_city' column
df['customer_city'] = df['customer_region'].apply(categorize_city)

print(df['customer_city'].value_counts(dropna=False))

In [ ]:
# Create the count plot
plt.figure(figsize=(8,6))
sns.countplot(x='customer_city', data=df, palette='tab10')

# Add titles and labels
plt.title('City', fontsize=14)
plt.xlabel('customer_city', fontsize=12)
plt.ylabel('Count', fontsize=12)

plt.show()

- Weekdays and Weekends

In [104]:
df['Weekdays'] = df[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']].sum(axis=1)
df['Weekends'] = df[['Saturday', 'Sunday']].sum(axis=1)

In [ ]:
# Reshape the data for easier plotting
df_melted = df[['Weekdays', 'Weekends']].melt(var_name='Day Type', value_name='Total')

# Create a count plot 
sns.barplot(data=df_melted, x='Day Type', y='Total', estimator=sum, ci=None, palette="tab10")

# Add labels and title
plt.title('Comparison of Total Values: Weekdays vs Weekends')
plt.xlabel('Day Type')
plt.ylabel('Total Value')


plt.show()

- Types of cuisine

In [ ]:
df['CUI_OTHER'].sum()

In [ ]:
(df['CUI_OTHER'] != 0).sum()

In [108]:
# Dictionary to group cuisines by style/type
groupings = {
    'Main Courses': [
        'CUI_American', 'CUI_Chicken Dishes', 'CUI_Indian', 
        'CUI_Italian', 'CUI_Thai', 'CUI_Chinese', 'CUI_Asian', 'CUI_Japanese'
    ],
    'Snacks and Street Food': [
        'CUI_Street Food / Snacks', 'CUI_Noodle Dishes', 'CUI_Cafe'
    ],
    'Desserts and Beverages': [
        'CUI_Desserts', 'CUI_Beverages'
    ],
    'Healthy and Special Diets': [
        'CUI_Healthy'
    ],
    'Other': [
        'CUI_OTHER'
    ]
}

# Function to aggregate money spent into cuisine groups
def aggregate_to_groups(df, groupings):
    # Create an empty DataFrame for the grouped values with the same index as df
    grouped_df = pd.DataFrame(index=df.index)
    
    # For each group, sum the relevant columns
    for group, cuisines in groupings.items():
        # Check if all columns in the group exist in the DataFrame
        if all(cuisine in df.columns for cuisine in cuisines):
            grouped_df[group] = df[cuisines].sum(axis=1)
        else:
            missing_columns = [cuisine for cuisine in cuisines if cuisine not in df.columns]
            print(f"Missing columns for group '{group}': {missing_columns}")
    
    # Return the DataFrame with the new group columns
    return grouped_df

# Aggregate the money spent into cuisine groups
grouped_df = aggregate_to_groups(df, groupings)

# Concatenate the grouped_df to the original df
df= pd.concat([df, grouped_df], axis=1)

In [ ]:
# Summing up the values for each group
group_totals = grouped_df.sum().sort_values(ascending=False)

# Plotting the aggregated values
plt.figure(figsize=(10, 6))
sns.barplot(x=group_totals.index, y=group_totals.values, palette="tab10")

# Adding labels and title
plt.title("Total Spending by Cuisine Group", fontsize=16)
plt.xlabel("Cuisine Group", fontsize=14)
plt.ylabel("Total Spending", fontsize=14)
plt.xticks(rotation=45, ha='right')  
plt.tight_layout()

plt.show()

<a class="anchor" id="visualization">

# 6. Visualizations
    
</a>

<a class="anchor" id="corr">

## 6.1 Correlations and Heatmaps
    
</a>

In [ ]:
df_corr = df[['customer_age', 
              'vendor_count',
               'product_count', 
              'first_order', 
              'last_order'
             ]].corr()
df_corr

In [ ]:
sns.heatmap(df_corr, 
            annot=True, 
            cmap='PiYG')

plt.title('Correlation Heatmap between numerical variables', fontsize=12)

- product_count and vendor_count have a very high correlation (0,86)

In [ ]:
df_corr_all=df[['customer_age', 'vendor_count',
       'product_count', 'first_order', 'last_order', 'CUI_American', 'CUI_Asian', 'CUI_Beverages',
       'CUI_Cafe', 'CUI_Chicken Dishes', 'CUI_Chinese', 'CUI_Desserts',
       'CUI_Healthy', 'CUI_Indian', 'CUI_Italian', 'CUI_Japanese',
       'CUI_Noodle Dishes', 'CUI_OTHER', 'CUI_Street Food / Snacks',
       'CUI_Thai', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday',
       'Saturday', 'Sunday', 'HR_0', 'HR_1', 'HR_2', 'HR_3', 'HR_4', 'HR_5',
       'HR_6', 'HR_7', 'HR_8', 'HR_9', 'HR_10', 'HR_11', 'HR_12', 'HR_13',
       'HR_14', 'HR_15', 'HR_16', 'HR_17', 'HR_18', 'HR_19', 'HR_20', 'HR_21',
       'HR_22', 'HR_23']].corr()

plt.figure(figsize=(20, 20)) 


sns.heatmap(df_corr_all, annot=True,
            linewidths=0.7, vmin=-1, vmax=1, square=True,
            cbar_kws={'shrink': 0.75, 'aspect': 30}, 
            annot_kws={'size': 6 },  
            cmap='PiYG')  

plt.xticks(rotation=45, ha='right', fontsize=12)
plt.yticks(rotation=0, fontsize=12)

plt.title('Correlation Heatmap with all numeric variables', fontsize=16, weight='bold')

plt.show()

In [ ]:
# Create an empty DataFrame to store the aggregated activity, initializing with zeros
heatmap_data = pd.DataFrame(index=dow_columns, columns=hr_columns)

# Sum of the hourly activity for each day and fill in the heatmap data
for day in dow_columns:
    # Summing the hourly columns for the current day and filling NaNs with zero
    heatmap_data.loc[day] = df.loc[df[day]  > 0, hr_columns].sum().fillna(0)

# Convert all data to numeric (float)
heatmap_data = heatmap_data.astype(float)

# Plot the heatmap
plt.figure(figsize=(30, 15))
sns.heatmap(heatmap_data, 
            cmap='YlGnBu', 
            linewidths=1, 
            annot=True, 
            square=True, 
            fmt='.0f')
plt.title('Heatmap of Hourly Activity Throughout the Week')
plt.xlabel('Hour of the Day (0-23)')
plt.ylabel('Day of the Week')
plt.show()

In [ ]:
# Create an empty DataFrame to store the aggregated activity, initializing with zeros
heatmap_data = pd.DataFrame(index=dow_columns, columns=time_columns)

# Sum the hourly activity for each day and fill in the heatmap data
for day in dow_columns:
    # Summing the hourly columns for the current day and filling NaNs with zero
    heatmap_data.loc[day] = df.loc[df[day] > 0, time_columns].sum().fillna(0)

# Convert all data to numeric (float)
heatmap_data = heatmap_data.astype(float)

# Plot the heatmap
plt.figure(figsize=(6, 6))
sns.heatmap(heatmap_data, 
            cmap='YlGnBu', 
            linewidths=0.5, 
            annot=True, 
            square=True,
            annot_kws={'size': 10 }, 
            fmt='.0f')
plt.title('Heatmap of Period of Time Activity Throughout the Week')
plt.xlabel('Period of Time')
plt.ylabel('Day of the Week')
plt.show()

In [ ]:
cuisine_by_age = df.groupby('age_group')[CUI_columns].sum()

plt.figure(figsize=(12, 8))
sns.heatmap(cuisine_by_age, 
            annot=True,
            annot_kws={'size': 10 }, 
            cmap='YlGnBu', 
            fmt='.0f', 
            square=True)

plt.title('Average Spend on each Cuisine by Age', fontsize=16)
plt.xlabel('Cuisine Type', fontsize=12)
plt.ylabel('Age Group', fontsize=12)


plt.xticks(rotation=45, ha='right')

plt.tight_layout()
plt.show()

In [ ]:
# Group by region and sum the DOW
cuisine_by_region = df.groupby('customer_region')[dow_columns].sum()  
plt.figure(figsize=(10, 6))
sns.heatmap(cuisine_by_region, 
            annot=True,
            annot_kws={'size': 10}, 
            cmap='YlGnBu', 
            fmt='.0f', 
            square=True)

plt.title('DOW Activity by Region', fontsize=16)
plt.xlabel('DOW', fontsize=12)  
plt.ylabel('Region', fontsize=12)

plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Group by city and sum the DOW
cuisine_by_region = df.groupby('customer_city')[dow_columns].sum()  
plt.figure(figsize=(10, 6))
sns.heatmap(cuisine_by_region, 
            annot=True,
            annot_kws={'size': 10}, 
            cmap='YlGnBu', 
            fmt='.0f', 
            square=True)

plt.title('DOW Activity by city', fontsize=16)
plt.xlabel('DOW', fontsize=12)  
plt.ylabel('city', fontsize=12)

plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

<a class="anchor" id="multi">

## 6.2 Multivariate Analysis
    
</a>

- Payment Method

In [ ]:
payment_counts = df.groupby(['age_group', 'payment_method']).size().unstack(fill_value=0)

colors=['#87CEEB','#00BFFF','#4682B4']
payment_counts.plot(kind='bar', 
                    stacked=False, 
                    figsize=(10, 6), 
                    color=colors, 
                    edgecolor='black') 

plt.title('Payment Methods by Age Group')
plt.xlabel('Age Group')
plt.ylabel('Number of orders')
plt.xticks(rotation=0)  
plt.legend(title='Payment Method')
plt.tight_layout()  
plt.show()

In [ ]:
payment_counts_region = df.groupby(['customer_region', 'payment_method']).size().unstack(fill_value=0)

colors=['#87CEEB','#00BFFF','#4682B4']
payment_counts_region.plot(kind='bar', 
                    stacked=False, 
                    figsize=(10, 6), 
                    color=plt.cm.tab10.colors, 
                    edgecolor='black') 

plt.title('Payment Methods by Region')
plt.xlabel('Region')
plt.ylabel('Number of orders')
plt.xticks(rotation=0)  
plt.legend(title='Payment Method')
plt.tight_layout()  
plt.show()

In [ ]:
payment_counts_region = df.groupby(['customer_city', 'payment_method']).size().unstack(fill_value=0)

colors=['#87CEEB','#00BFFF','#4682B4']
payment_counts_region.plot(kind='bar', 
                    stacked=False, 
                    figsize=(10, 6), 
                    color=plt.cm.tab10.colors, 
                    edgecolor='black') 

plt.title('Payment Methods by City')
plt.xlabel('City')
plt.ylabel('Number of orders')
plt.xticks(rotation=0)  
plt.legend(title='Payment Method')
plt.tight_layout()  
plt.show()

- Age Group

In [ ]:
# Grouping data by region and age group
age_counts = df.groupby(['customer_region', 'age_group']).size().reset_index(name='count')

# Creating a pivot table
pivot_age_counts = age_counts.pivot(index='customer_region', columns='age_group', values='count').fillna(0)

# Plotting the stacked bar chart
pivot_age_counts.plot(kind='bar', 
                      stacked=True, 
                      figsize=(10, 6), 
                      color=plt.cm.tab10.colors) # Ensuring the color palette is correct

plt.title('Distribution of Age Groups by Region')
plt.xlabel('Region')
plt.ylabel('Number of Customers')
plt.xticks(rotation=45)  
plt.legend(title='Age Group', bbox_to_anchor=(1.05, 1), loc='upper left')  
plt.tight_layout()  
plt.show()

In [ ]:
# Grouping data by city and age group
age_counts_city = df.groupby(['customer_city', 'age_group']).size().reset_index(name='count')

# Create the stacked bar plot
age_counts_city_pivot = age_counts_city.pivot(index='customer_city', columns='age_group', values='count').fillna(0)

age_counts_city_pivot.plot(kind='bar', 
                            stacked=True, 
                            figsize=(10, 6),
                            color=plt.cm.tab10.colors) # Ensuring the color palette is correct
plt.title('Distribution of Age Groups by City')
plt.xlabel('City')
plt.ylabel('Number of Customers')
plt.xticks(rotation=45)  
plt.legend(title='Age Group', bbox_to_anchor=(1.05, 1), loc='upper left')  
plt.tight_layout()  
plt.show()

- Cuisine Type

In [ ]:
cuisine_counts = df.groupby('customer_region')[CUI_columns].sum().reset_index()

cuisine_counts_ = cuisine_counts.melt(id_vars='customer_region', 
                                            value_vars=CUI_columns, 
                                            var_name='cuisine_type', 
                                            value_name='total_expenditure')

top_cuisines = (
    cuisine_counts_.groupby('customer_region')
    .apply(lambda x: x.nlargest(3, 'total_expenditure'))
    .reset_index(drop=True)
)


top_cuisine_counts = top_cuisines.pivot(index='customer_region', 
                                        columns='cuisine_type', 
                                        values='total_expenditure').fillna(0)


num_cuisines = top_cuisine_counts.shape[1]
colors = plt.cm.Paired(np.linspace(0, 1, num_cuisines)) 


top_cuisine_counts.plot(kind='bar', 
                        stacked=True, 
                        figsize=(10, 6), 
                        color=colors)

plt.title('Top 3 Cuisine Type by Region')
plt.xlabel('Region')
plt.ylabel('Total Expenditure')
plt.xticks(rotation=45)
plt.legend(title='Type of Cuisine', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
cuisine_counts = df.groupby('customer_city')[CUI_columns].sum().reset_index()

cuisine_counts_1= cuisine_counts.melt(id_vars='customer_city', 
                                            value_vars=CUI_columns, 
                                            var_name='cuisine_type', 
                                            value_name='total_expenditure')

top_cuisines = (
    cuisine_counts_1.groupby('customer_city')
    .apply(lambda x: x.nlargest(3, 'total_expenditure'))
    .reset_index(drop=True)
)


top_cuisine_counts = top_cuisines.pivot(index='customer_city', 
                                        columns='cuisine_type', 
                                        values='total_expenditure').fillna(0)


num_cuisines = top_cuisine_counts.shape[1]
colors = plt.cm.Paired(np.linspace(0, 1, num_cuisines)) 


top_cuisine_counts.plot(kind='bar', 
                        stacked=True, 
                        figsize=(10, 6), 
                        color=colors)

plt.title('Top 3 Cuisine Type by City')
plt.xlabel('City')
plt.ylabel('Total Expenditure')
plt.xticks(rotation=45)
plt.legend(title='Type of Cuisine', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


- Total Spending

In [ ]:
# Group by region and sum the total spending
region_spend = df.groupby('customer_region')['total_spend'].sum().reset_index()

region_spend = region_spend.sort_values(by='total_spend', ascending=False)

# Plotting
plt.figure(figsize=(10, 6))
sns.barplot(x='customer_region', y='total_spend', data=region_spend, palette="tab10")

plt.title("Total Spending per Region")
plt.xlabel("Customer Region")
plt.ylabel("Total Spend")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Group by age group and  total spending
region_spend = df.groupby('customer_city')['total_spend'].sum().reset_index()

region_spend = region_spend.sort_values(by='total_spend', ascending=False)

# Plotting
plt.figure(figsize=(10, 6))
sns.barplot(x='customer_city', y='total_spend', data=region_spend, palette="tab10")

plt.title("Total Spending per City")
plt.xlabel("City")
plt.ylabel("Total Spend")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Group by age group and  total spending
region_spend = df.groupby('age_group')['total_spend'].sum().reset_index()

region_spend = region_spend.sort_values(by='total_spend', ascending=False)

# Plotting
plt.figure(figsize=(10, 6))
sns.barplot(x='age_group', y='total_spend', data=region_spend, palette="tab10")

plt.title("Total Spending per Age Group")
plt.xlabel("Age Group")
plt.ylabel("Total Spend")
plt.xticks(rotation=45)
plt.show()

- Cuisine Diversity

In [ ]:
# Group by region and cuisine diversity
region_spend = df.groupby('customer_region')['cuisine_diversity'].sum().reset_index()

region_spend = region_spend.sort_values(by='cuisine_diversity', ascending=False)

# Plotting
plt.figure(figsize=(10, 6))
sns.barplot(x='customer_region', y='cuisine_diversity', data=region_spend, palette="tab10")

plt.title("Cuisine Diversity per Region")
plt.xlabel("Customer Region")
plt.ylabel("Cuisine Diveristy")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Group by region and cuisine diversity
region_spend = df.groupby('customer_city')['cuisine_diversity'].sum().reset_index()

region_spend = region_spend.sort_values(by='cuisine_diversity', ascending=False)

# Plotting
plt.figure(figsize=(10, 6))
sns.barplot(x='customer_city', y='cuisine_diversity', data=region_spend, palette="tab10")

plt.title("Cuisine Diversity per city")
plt.xlabel("Customer city")
plt.ylabel("Cuisine Diveristy")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Group by age group and cuisine diversity
region_spend = df.groupby('age_group')['cuisine_diversity'].sum().reset_index()

region_spend = region_spend.sort_values(by='cuisine_diversity', ascending=False)

# Plotting
plt.figure(figsize=(10, 6))
sns.barplot(x='age_group', y='cuisine_diversity', data=region_spend, palette="tab10")

plt.title("Cuisine Diversity per Age Group")
plt.xlabel("Age Group")
plt.ylabel("Cuisine Diversity")
plt.xticks(rotation=45)
plt.show()

<a class="anchor" id="enc">

# 7. Encoding
    
</a>

In [ ]:
df.columns.values

In [135]:
df = df.drop(columns=['customer_region'])

In [136]:
df_ohc = df.copy()

In [137]:
categorical_features_new = ['last_promo', 'payment_method', 'age_group', 'is_chain', 'customer_city']

In [138]:
# Remove 'age_group' and 'last_promo' from the list of categorical features
columns_to_remove = ['age_group', 'last_promo']  
cf_for_ohc = [col for col in categorical_features_new if col not in columns_to_remove]

# Initialize the encoder with the chosen settings
ohc = OneHotEncoder(sparse_output=False, drop=None)

# Fit the encoder on the selected categorical features
ohc.fit(df_ohc[cf_for_ohc])

# Transform the selected categorical features
ohc_features = ohc.transform(df_ohc[cf_for_ohc])

# Create a DataFrame with encoded feature names, adding 'enc_' prefix to the columns
ohc_df = pd.DataFrame(
    ohc_features,
    index=df_ohc.index,
    columns=[f"enc_{name}" for name in ohc.get_feature_names_out(cf_for_ohc)]
)

# Concatenate the encoded features back to the original dataset (without dropping the original columns)
ohc_encoded = pd.concat(
    [df_ohc, ohc_df],
    axis=1
)

In [ ]:
print(ohc_encoded.head())

In [140]:
# Define the encoder for 'age_group' with categories
ordinal_encoder = OrdinalEncoder(
    categories=[['Teenagers (15-19)', 'Young Adults (20-29)', 'Adults (30-49)', 'Middle-aged (50-64)', 'Seniors (65-80)']],
    handle_unknown='use_encoded_value',
    unknown_value=-1  # Use -1 for unknown categories
)

if 'age_group' not in df_ohc.columns:
    raise KeyError("The column 'age_group' is not in the DataFrame.")

# Fit and transform the 'age_group' column and add it to the DataFrame with prefix 'enc_'
df_ohc['enc_age_group'] = ordinal_encoder.fit_transform(df_ohc[['age_group']])

# Map the 'last_promo' feature to binary values
promo_mapping = {
    'No PROMO': 0,  # No promo maps to 0
    'DISCOUNT': 1,   # All promos map to 1
    'DELIVERY': 1,
    'FREEBIE': 1
}

# Apply the mapping to create the binary encoded column for 'last_promo' with prefix 'enc_'
df_ohc['enc_last_promo'] = df_ohc['last_promo'].map(promo_mapping)

# If there are any NaN values in 'enc_last_promo', fill them with 0
df_ohc['enc_last_promo'] = df_ohc['enc_last_promo'].fillna(0)

# Concatenate the encoded columns with the existing DataFrame
df_encoded = pd.concat([ohc_encoded, df_ohc[['enc_age_group', 'enc_last_promo']]], axis=1)

In [ ]:
print(df_encoded.head())

In [142]:
df = df_encoded

<a class="anchor" id="scale">

# 8. Scaling
    
</a>

<a class="anchor" id="mm">

## 8.1 MinMax Scaling
    
</a>

In [143]:
df_minmax = df.copy()

In [ ]:
df.columns.values

In [145]:
numeric_cols =['customer_age', 'vendor_count', 'product_count','first_order', 'last_order',
        'CUI_American', 'CUI_Asian', 'CUI_Beverages', 'CUI_Cafe',
        'CUI_Chicken Dishes', 'CUI_Chinese', 'CUI_Desserts', 'CUI_Healthy',
        'CUI_Indian', 'CUI_Italian', 'CUI_Japanese', 'CUI_Noodle Dishes',
        'CUI_OTHER', 'CUI_Street Food / Snacks', 'CUI_Thai', 'Sunday',
        'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday',
        'HR_0', 'HR_1', 'HR_2', 'HR_3', 'HR_4', 'HR_5', 'HR_6', 'HR_7',
        'HR_8', 'HR_9', 'HR_10', 'HR_11', 'HR_12', 'HR_13', 'HR_14',
        'HR_15', 'HR_16', 'HR_17', 'HR_18', 'HR_19', 'HR_20', 'HR_21',
        'HR_22', 'HR_23', 'early_morning(0h-5h)', 'morning(6h-11h)',
        'afternoon(12h-17h)', 'night(18h-23h)','Sum_of_Orders', 'recency', 'active_period', 'frequency',
        'total_spend', 'cuisine_diversity','Weekends', 'Main Courses', 'Snacks and Street Food',
       'Desserts and Beverages', 'Healthy and Special Diets', 'Other']

In [ ]:
# Use MinMaxScaler to scale the data
mm_scaler = MinMaxScaler()
mm_scaled_feat = mm_scaler.fit_transform(df_minmax[numeric_cols])
mm_scaled_feat

In [ ]:
# See what the fit method is doing:
print("Parameters fitted:\n")
pd.DataFrame([mm_scaler.data_min_, mm_scaler.data_max_], columns=numeric_cols, index=['min','max'])

In [148]:
## Replace original numeric_cols values with mm_scaled_feat values
df_minmax[numeric_cols] = mm_scaled_feat

In [ ]:
# Checking max and min of minmaxed variables
df_minmax[numeric_cols].describe().round(2)

<a class="anchor" id="std">

## 8.2 Standard Scaling
    
</a>

In [150]:
df_standard = df.copy()

In [ ]:
ss_scaler = StandardScaler()
ss_scaled_feat = ss_scaler.fit_transform(df_standard[numeric_cols])
ss_scaled_feat

In [ ]:
# See what the fit method is doing:
print("Parameters fitted:\n")
pd.DataFrame([ss_scaler.mean_, np.sqrt(ss_scaler.var_)], columns=numeric_cols, index=['mean','std'])

In [ ]:
df_standard[numeric_cols] = ss_scaled_feat
df_standard.head()

In [ ]:
# Checking mean and variance of standardized variables
df_standard[numeric_cols].describe().round(2)

<a class="anchor" id="vs">

## 8.3 MinMaxScaler vs StandardScaler vs Original data
    
</a>

<a class="anchor" id="box">

### 8.3.1 Boxplots
    
</a>

In [ ]:
sns.set_style('whitegrid')

# Grid
fig, axes = plt.subplots(
    len(numeric_cols), 3, 
    figsize=(10, len(numeric_cols) * 3), 
    tight_layout=True, 
    sharex='col', 
    sharey='row'
)

for i, col in enumerate(numeric_cols):
    # Original
    sns.boxplot(df, x=col, ax=axes[i, 0], width=.4, color='steelblue')
    axes[i, 0].set_title('Original')
    axes[i, 0].set_ylabel(col)

    # MinMaxScaler
    sns.boxplot(df_minmax, x=col, ax=axes[i, 1], width=.4, color='steelblue')
    axes[i, 1].set_title('MinMaxScaler()')

    # StandardScaler
    sns.boxplot(df_standard, x=col, ax=axes[i, 2], width=.4, color='steelblue')
    axes[i, 2].set_title('StandardScaler()')


    axes[i, 0].set_xlabel(None)
    axes[i, 1].set_xlabel(None)
    axes[i, 2].set_xlabel(None)

fig.suptitle('Boxplots', y=1.02)

plt.show()


<a class="anchor" id="histograms">

### 8.3.2 Histograms
    
</a>

In [ ]:
sns.set_style('whitegrid')

# Grid for plots
fig, axes = plt.subplots(
    len(numeric_cols), 3, 
    figsize=(10, len(numeric_cols) * 3), 
    tight_layout=True, 
    sharex='col', 
    sharey='row'
)

for i, col in enumerate(numeric_cols):
    # Original
    sns.histplot(df, x=col, ax=axes[i, 0], binwidth=1, kde=False, color='steelblue')
    axes[i, 0].set_title('Original')
    axes[i, 0].set_ylabel(col)

    # MinMaxScaler
    sns.histplot(df_minmax, x=col, ax=axes[i, 1], binwidth=0.1, kde=False)
    axes[i, 1].set_title('MinMaxScaler()')

    # StandardScaler
    sns.histplot(df_standard, x=col, ax=axes[i, 2], binwidth=0.1, kde=False)
    axes[i, 2].set_title('StandardScaler()')

    axes[i, 0].set_xlabel(None)
    axes[i, 1].set_xlabel(None)
    axes[i, 2].set_xlabel(None)

fig.suptitle('Histograms', y=1.02)

plt.show()


Use StandardScaler:

In [160]:
df = df_standard.copy()

In [ ]:
df.head()

In [162]:
#Save dataset before clustering
df.to_csv('data_beforeclustering.csv')
